# Postprocess the raw simulation data

Go through all subdirectories with name `out/T_*`, and use simulation data to compute overview of Mr, Hc and BHmax as function of temperature. Compute Hc, Mr, and BHmax at this point.

In [1]:
import json
import pathlib

import mammos_analysis
import mammos_entity as me
import pandas as pd

In [2]:
Ts = []
Hc = []
Mr = []
BHmax = []

In [3]:
runs = list(pathlib.Path("./out").glob(pattern="T_*"))

In [4]:
runs

[PosixPath('out/T_941.0067172639365'),
 PosixPath('out/T_0.0'),
 PosixPath('out/T_67.2147655188526'),
 PosixPath('out/T_201.6442965565578'),
 PosixPath('out/T_806.5771862262312'),
 PosixPath('out/T_268.8590620754104'),
 PosixPath('out/T_537.7181241508208'),
 PosixPath('out/T_672.147655188526'),
 PosixPath('out/T_873.7919517450838'),
 PosixPath('out/T_470.50335863196824'),
 PosixPath('out/T_604.9328896696734'),
 PosixPath('out/T_403.2885931131156'),
 PosixPath('out/T_134.4295310377052'),
 PosixPath('out/T_336.073827594263'),
 PosixPath('out/T_1008.2214827827889'),
 PosixPath('out/T_739.3624207073786')]

In [5]:
for run in runs:
    print(f"Working on {run=}")
    T = float(str(run).split("T_")[1])
    try:
        df = pd.read_csv(run / "hystloop.dat", sep="\t")
    except FileNotFoundError:
        # if material is too soft, hystloop.dat is not written
        print(f"No hystloop.dat found for {T=}")
        continue

    file = run / "parameters.json"
    # sanity check: is the temperature that the simulation saved the same as in the directory name?
    with open(file) as handle:
        param_dict = json.load(handle)
        assert param_dict["T"] == T

    # compute extrinsic parameters
    H = df["H (A/m)"].values
    M = df["M (A/m)"].values
    res = mammos_analysis.hysteresis.extrinsic_properties(H=me.H(H), M=me.M(M), demagnetization_coefficient=1 / 3)

    # collect extrinsic parameters
    Ts.append(T)
    Hc.append(res.Hc.value)
    Mr.append(res.Mr.value)
    BHmax.append(res.BHmax.value)

Working on run=PosixPath('out/T_941.0067172639365')
Working on run=PosixPath('out/T_0.0')
Working on run=PosixPath('out/T_67.2147655188526')
Working on run=PosixPath('out/T_201.6442965565578')
Working on run=PosixPath('out/T_806.5771862262312')
Working on run=PosixPath('out/T_268.8590620754104')
Working on run=PosixPath('out/T_537.7181241508208')
Working on run=PosixPath('out/T_672.147655188526')
Working on run=PosixPath('out/T_873.7919517450838')
Working on run=PosixPath('out/T_470.50335863196824')
Working on run=PosixPath('out/T_604.9328896696734')
Working on run=PosixPath('out/T_403.2885931131156')
Working on run=PosixPath('out/T_134.4295310377052')
Working on run=PosixPath('out/T_336.073827594263')
Working on run=PosixPath('out/T_1008.2214827827889')
Working on run=PosixPath('out/T_739.3624207073786')


In [6]:
# put data into DataFrame for easy sorting according to temperature T
parameters_df = pd.DataFrame(
    {
        "T": Ts,
        "Hc": Hc,
        "Mr": Mr,
        "BHmax": BHmax,
    }
)

parameters_df.sort_values("T", inplace=True, ignore_index=True)

In [7]:
## Write data to disk as entity collection.
me.io.entities_to_file(
    "hystloop_extrinsic_properties.csv",
    "These extrinsic properties were calculated using mammos-mumag simulations with 300 steps.",
    T=me.T(parameters_df["T"]),
    Hc=me.Hc(parameters_df["Hc"]),
    Mr=me.Mr(parameters_df["Mr"]),
    BHmax=me.BHmax(parameters_df["BHmax"]),
)

In [8]:
mammos_analysis.__version__

'0.4.0'